In [456]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [457]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [458]:
len(words)

32033

In [459]:
# build the vocab of chars and mapping to/from ints
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [460]:
# build the dataset
block_size = 3 # context length: how many chars we take to predict the next?
X, Y = [], []
for w in words[:5]:
    print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '---->', itos[ix])
        context = context[1:] + [ix] # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ----> e
..e ----> m
.em ----> m
emm ----> a
mma ----> .
olivia
... ----> o
..o ----> l
.ol ----> i
oli ----> v
liv ----> i
ivi ----> a
via ----> .
ava
... ----> a
..a ----> v
.av ----> a
ava ----> .
isabella
... ----> i
..i ----> s
.is ----> a
isa ----> b
sab ----> e
abe ----> l
bel ----> l
ell ----> a
lla ----> .
sophia
... ----> s
..s ----> o
.so ----> p
sop ----> h
oph ----> i
phi ----> a
hia ----> .


In [461]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [462]:
C = torch.randn((27,2))

In [463]:
C[5]

tensor([0.6591, 0.2168])

In [464]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [465]:
#torch.cat(torch.unbind(emb, 1), 1).shape # this is inefficient vs .view() because it creates a new tensor after concatinating taking up memory
#emb.view(32, 6) == torch.cat(torch.unbind(emb, 1), 1) this comparison returns true showing view functions the same as cat and unbind while reducing memory use. view is just more efficient

In [466]:
W1 = torch.randn((6,100))
b1 = torch.randn(100)

In [467]:
# instead of emb.shape[0] you can put -1 and pytorch will infer the shape
h = torch.tanh(emb.view(emb.shape[0], 6) @ W1 + b1)

In [468]:
h

tensor([[ 0.1808, -0.8339, -1.0000,  ..., -0.5237, -0.7594, -0.9639],
        [ 0.5301, -0.9908, -0.9918,  ..., -0.4493,  0.1278, -0.8463],
        [-0.9029, -0.9505, -0.8403,  ...,  0.9755,  0.5898, -0.4752],
        ...,
        [-0.2540,  0.8406, -0.9977,  ..., -0.9987, -0.2895,  0.6153],
        [ 0.8933, -0.9430, -0.9984,  ..., -0.9491,  0.9761,  0.8058],
        [-0.2590, -0.9734, -0.9827,  ...,  0.9728,  0.9935,  0.4028]])

In [469]:
h.shape

torch.Size([32, 100])

In [470]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [471]:
logits = h @ W2 + b2

In [472]:
logits.shape

torch.Size([32, 27])

In [473]:
counts = logits.exp()

In [474]:
prob = counts / counts.sum(1, keepdim=True)

In [475]:
prob.shape

torch.Size([32, 27])

In [476]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(16.2389)

In [477]:
# reorganize everything above-------------------------------------------------------------------------------------------------

In [478]:
X.shape, Y.shape # dataset

(torch.Size([32, 3]), torch.Size([32]))

In [479]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27,2), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [480]:
sum(p.nelement() for p in parameters) # num params in total

3481

In [482]:
emb = C[X] # (32, 3, 2)
h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # (32, 100)
logits = h @ W2 + b2 # (32, 27)
counts = logits.exp()
prob = counts / counts.sum(1, keepdim=True)
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(17.7697)